# Applied Computational Intelligence - HW2

Authors
- Brewton Morais
- Lucas Abdalah

In [44]:
# Packages and Data Import
from tkinter import OFF
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
from scipy.stats import zscore, norm, linregress
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cbook import boxplot_stats

# Solubility data divided into train and test sets
X_test = pd.read_csv(r"https://raw.githubusercontent.com/lucasabdalah/Exploratory-Data-Analisys/hw2-newcode/code/hw2/solTestX.csv")
X_train = pd.read_csv(r"https://raw.githubusercontent.com/lucasabdalah/Exploratory-Data-Analisys/hw2-newcode/code/hw2/solTrainX.csv")
Y_train = pd.read_csv(r"https://raw.githubusercontent.com/lucasabdalah/Exploratory-Data-Analisys/hw2-newcode/code/hw2/solTrainY.csv")
Y_test = pd.read_csv(r"https://raw.githubusercontent.com/lucasabdalah/Exploratory-Data-Analisys/hw2-newcode/code/hw2/solTestY.csv")




In [45]:
# concatenate data
X= pd.concat([X_train, X_test], axis=0)
Y = pd.concat([Y_train, Y_test], axis = 0)

# Target: solubility of chemical compound
data = pd.concat([X, Y], axis = 1)
data = data.rename(columns={'x':'solubility'})

In [46]:
data.head(5)

,FP001,FP002,FP003,FP004,FP005,FP006,FP007,FP008,FP009,FP010,...,NumNitrogen,NumOxygen,NumSulfer,NumChlorine,NumHalogen,NumRings,HydrophilicFactor,SurfaceArea1,SurfaceArea2,solubility
0,0,1,0,0,1,0,0,1,0,0,...,2,0,0,0,0,3,-0.856,25.78,25.78,-3.97
1,0,1,0,1,1,1,1,1,0,0,...,3,1,1,0,0,4,-0.370,52.19,80.43,-3.98
2,1,1,1,1,1,0,0,1,0,1,...,0,2,0,0,0,1,-0.330,37.30,37.30,-3.99
3,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,1,-0.960,0.00,0.00,-4.00
4,0,0,1,1,1,1,0,0,1,0,...,5,0,0,1,1,1,-0.069,53.94,53.94,-4.06


In [47]:
data.describe()

,FP001,FP002,FP003,FP004,FP005,FP006,FP007,FP008,FP009,FP010,...,NumNitrogen,NumOxygen,NumSulfer,NumChlorine,NumHalogen,NumRings,HydrophilicFactor,SurfaceArea1,SurfaceArea2,solubility
count,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,...,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000,1267.000000
mean,0.486977,0.550908,0.438043,0.571429,0.588792,0.388319,0.361484,0.334649,0.275454,0.182320,...,0.786898,1.528019,0.148382,0.556433,0.700868,1.400947,-0.040946,35.537238,38.939013,-2.738137
std,0.500028,0.497598,0.496342,0.495067,0.492247,0.487560,0.480620,0.472054,0.446919,0.386261,...,1.171159,1.679670,0.460188,1.400382,1.475880,1.281266,1.092327,34.409726,37.047585,2.054083
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.986000,0.000000,0.000000,-11.620000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.766500,9.230000,9.230000,-3.955000
50%,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,-0.338000,27.690000,32.260000,-2.490000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,1.000000,2.000000,0.000000,0.000000,1.000000,2.000000,0.312000,52.320000,57.200000,-1.360000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,6.000000,13.000000,4.000000,10.000000,10.000000,7.000000,13.483000,331.940000,331.940000,1.580000


In [48]:
print("N = {} Observations\n".format(len(data.axes[0])))
print("D = {} Predictor variables\n".format(len(data.columns)))
print("Predictor Names:\n{}\n".format(data.columns.to_list()))
print("Class-Counts:\n{}".format(data["solubility"].value_counts()))

N = 1267 Observations

D = 229 Predictor variables

Predictor Names:
['FP001', 'FP002', 'FP003', 'FP004', 'FP005', 'FP006', 'FP007', 'FP008', 'FP009', 'FP010', 'FP011', 'FP012', 'FP013', 'FP014', 'FP015', 'FP016', 'FP017', 'FP018', 'FP019', 'FP020', 'FP021', 'FP022', 'FP023', 'FP024', 'FP025', 'FP026', 'FP027', 'FP028', 'FP029', 'FP030', 'FP031', 'FP032', 'FP033', 'FP034', 'FP035', 'FP036', 'FP037', 'FP038', 'FP039', 'FP040', 'FP041', 'FP042', 'FP043', 'FP044', 'FP045', 'FP046', 'FP047', 'FP048', 'FP049', 'FP050', 'FP051', 'FP052', 'FP053', 'FP054', 'FP055', 'FP056', 'FP057', 'FP058', 'FP059', 'FP060', 'FP061', 'FP062', 'FP063', 'FP064', 'FP065', 'FP066', 'FP067', 'FP068', 'FP069', 'FP070', 'FP071', 'FP072', 'FP073', 'FP074', 'FP075', 'FP076', 'FP077', 'FP078', 'FP079', 'FP080', 'FP081', 'FP082', 'FP083', 'FP084', 'FP085', 'FP086', 'FP087', 'FP088', 'FP089', 'FP090', 'FP091', 'FP092', 'FP093', 'FP094', 'FP095', 'FP096', 'FP097', 'FP098', 'FP099', 'FP100', 'FP101', 'FP102', 'FP103', 'FP

In [49]:
# Checking the existance of null values
data.isnull()

,FP001,FP002,FP003,FP004,FP005,FP006,FP007,FP008,FP009,FP010,...,NumNitrogen,NumOxygen,NumSulfer,NumChlorine,NumHalogen,NumRings,HydrophilicFactor,SurfaceArea1,SurfaceArea2,solubility
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
312,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
313,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
314,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [53]:
print("Predictor Mean: \n{}\n".format(data.mean(axis = 0).to_dict()))
print("Predictor Standard Deviation: \n{}\n".format(data.std(axis = 0).to_dict()))
print("Predictor Skewness: \n{}\n".format(data.skew(axis = 0).to_dict()))

Predictor Mean: 
{'FP001': 0.48697711128650356, 'FP002': 0.5509076558800315, 'FP003': 0.43804262036306235, 'FP004': 0.5714285714285714, 'FP005': 0.5887924230465666, 'FP006': 0.388318863456985, 'FP007': 0.36148382004735596, 'FP008': 0.3346487766377269, 'FP009': 0.27545382794001577, 'FP010': 0.18232044198895028, 'FP011': 0.20441988950276244, 'FP012': 0.17442778216258878, 'FP013': 0.16574585635359115, 'FP014': 0.16022099447513813, 'FP015': 0.8524072612470402, 'FP016': 0.1452249408050513, 'FP017': 0.1452249408050513, 'FP018': 0.11996842936069456, 'FP019': 0.11996842936069456, 'FP020': 0.11681136543014996, 'FP021': 0.11760063141278611, 'FP022': 0.10734017363851618, 'FP023': 0.11365430149960537, 'FP024': 0.10891870560378848, 'FP025': 0.10655090765588003, 'FP026': 0.09234411996842937, 'FP027': 0.0931333859510655, 'FP028': 0.09392265193370165, 'FP029': 0.08918705603788477, 'FP030': 0.09076558800315707, 'FP031': 0.09155485398579322, 'FP032': 0.08760852407261246, 'FP033': 0.08208366219415943, 'F

In [51]:
# Apply Normalization 
df = data.copy()
df = df.apply(zscore)

,FP001,FP002,FP003,FP004,FP005,FP006,FP007,FP008,FP009,FP010,...,NumNitrogen,NumOxygen,NumSulfer,NumChlorine,NumHalogen,NumRings,HydrophilicFactor,SurfaceArea1,SurfaceArea2,solubility
0,-0.974285,0.902877,-0.882890,-1.154701,0.835698,-0.796768,-0.752417,1.410038,-0.616583,-0.472200,...,1.036222,-0.910073,-0.322565,-0.397500,-0.475069,1.248519,-0.746458,-0.283672,-0.355332,-0.599951
1,-0.974285,0.902877,-0.882890,0.866025,0.835698,1.255071,1.329051,1.410038,-0.616583,-0.472200,...,1.890414,-0.314483,1.851319,-0.397500,-0.475069,2.029305,-0.301361,0.484146,1.120380,-0.604821
2,1.026394,0.902877,1.132644,0.866025,0.835698,-0.796768,-0.752417,1.410038,-0.616583,2.117746,...,-0.672162,0.281107,-0.322565,-0.397500,-0.475069,-0.313054,-0.264727,0.051249,-0.044258,-0.609692
3,-0.974285,-1.107571,1.132644,-1.154701,-1.196604,-0.796768,1.329051,-0.709201,-0.616583,-0.472200,...,-0.672162,-0.910073,-0.322565,-0.397500,-0.475069,-0.313054,-0.841705,-1.033175,-1.051469,-0.614562
4,-0.974285,-1.107571,1.132644,0.866025,0.835698,1.255071,-0.752417,-0.709201,1.621842,-0.472200,...,3.598799,-0.910073,-0.322565,0.316873,0.202760,-0.313054,-0.025693,0.535024,0.405071,-0.643784
